In [389]:
import pandas as pd
import math
file = pd.read_csv('45826522P9.txt',header = None)

In [393]:
# testfile = pd.read_csv('58814073S9.txt',header=None)   OK!

In [522]:
testfile = pd.read_csv('43838829N2.txt',header=None)

In [523]:
testfile
file = testfile

In [170]:
#回傳一個陣列，裡面有  除了第一列(也就是時間列)的所有value count
#呼叫時parameters放入要讀取的檔案
def countValues(file):
    file_header = list(file)
    unique_count = {}
    for i in range(1,len(file_header)):  #刻意不算時間欄位
        valueCounts = file[file_header[i]].value_counts()
        for j in range(len(valueCounts.index)):
            unique_count[valueCounts.index[j]] = valueCounts.values[j]
    return unique_count

#以下沒用到，以後要用可以參考
#unique_names = file[file_header[i]].value_counts().index.tolist() #算出個欄位unique值
#unique_count[valueCounts.indexs] = valueCounts.values
        

In [171]:
countValuesList = countValues(file)

In [172]:
def activityCount(file):  #使用這個function請先確定資料是照日期排序，且第二欄位為TrafficSource
    sessionJumpCount = 0 #算session跳另一組的次數
    rowCount = 0
    for i in range(len(file.index)):
        if i==0: 
            rowCount =1 
            continue
        if file[2][i] == file[2][i-1]: #如果session連續
            rowCount += 1
        else:
            #print("換另一session")
            sessionJumpCount += 1
            rowCount +=1
            
    print("rowCount = " + str(rowCount))
    print("sessionJumpCount =" +str(sessionJumpCount))
    # print( math.log2(rowCount) + sessionJumpCount) #97
    return math.sqrt(rowCount)+sessionJumpCount

In [173]:
activityCount(file)

rowCount = 10173
sessionJumpCount =1045


1145.8612908900138

In [174]:
def purchaseActivityCount(countValuesList): #執行過countValues函數，把那個函數的return值丟進來才能算
    purchaseActivityDict = {}
    favCount = countValuesList['Fav']
    cartCount = countValuesList['Cart']
    purchaseCount = countValuesList['Purchase']
    purchaseCartConversionRate = purchaseCount/cartCount # 購物次數/cart次數
    favPurchaseConversionRate = favCount/purchaseCount # fav次數/購物次數
    #所以purchaseCartConversionRate*favCartConversionRate =  fav次數/cart次數
    favCartConversionRate = purchaseCartConversionRate*favPurchaseConversionRate # fav/Cart
    purchaseActivityDict.update({'favCount' : favCount,
                                'cartCount' : cartCount,
                                'purchaseCount' : purchaseCount,
                                'purchaseCartConversionRate' : purchaseCartConversionRate,
                                'favPurchaseConversionRate' : favPurchaseConversionRate,
                                'favCartConversionRate' : favCartConversionRate})
    return purchaseActivityDict

In [175]:
purchaseActivityCount(countValuesList)

{'favCount': 37,
 'cartCount': 142,
 'purchaseCount': 64,
 'purchaseCartConversionRate': 0.4507042253521127,
 'favPurchaseConversionRate': 0.578125,
 'favCartConversionRate': 0.2605633802816901}

In [12]:
# print(file[3][1])

ViewSalePage


In [ ]:
# for i, row in enumerate(file.values):
#     print(i)
#     print(type(row[0]))
#     print(file.values[i-1])

In [ ]:
# for i, row in enumerate(file.values):
#     #print((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds())
#     print(i)
#     if(i%3==0):
#         i+=2

In [374]:
# print(len(file.index))

49


In [429]:
# txt = "8888 55 44"
# txt.split(" ")
#print(file[0][0].split(" ")[0])

Timestamp('2018-04-08 19:36:45')

In [527]:
import datetime
def purchaseBeforeCount(file):
    
    one_empty_row = pd.DataFrame([0])
    file = file.append(one_empty_row,ignore_index=True)
    
    #print(file)
    
    purchase_segment_dict = {}
    a_purchase_segment_row = 0
    a_purchase_segment_product_list = {}
    a_purchase_segment_source_type = {}
    a_purchase_segment_SessionCount=0   #　session的跳動count
    a_purchase_segment_SessionSeriesCount = 1 # session序列跳下一個session序列的跳動count
#     a_purchase_segment_SessionCount_totaltime = 0 # 每一個session 在一個purchase segment下的 總瀏覽時間
#     a_purchase_segment_SessionSeriesCount_totaltime = 0 # 每一個sessionSeries 在一個purchase segment下的 總瀏覽時間
    
    a_purchase_segment_behavior_type ={}
    a_purchase_segment_traffic_source ={}
    a_purchase_order = {} #訂單的東西
    purchase_orders = []
    purchase_order_serial_number = 0 #訂單編號
    before_purchase_counts=[]
    i=0
    
#     session_count_index = [] #儲存session count的初始row index 以及跳動發生的時候 之index
#     session_series_count_index = [] # 儲存session series count跟上述一樣的東西 目前還不確定正不正確
    
    
#    behavior_name =''
    
    purchase_amount=0
    discount_amount=0
    
#     sessiontime = pd.to_datetime(file.values[0][0])
#     session_count_index.append(0)
    for i, row in enumerate(file.values):
        
        if(row[3]!='Purchase' and row[3]!='Offline'):
            a_purchase_segment_row += 1
            try:
                a_purchase_segment_source_type["{}".format(row[4])] = a_purchase_segment_source_type["{}".format(row[4])]+1
            except:
                a_purchase_segment_source_type.update({"{}".format(row[4]) : 1})
            try:
                a_purchase_segment_product_list["{}".format(row[5])] = a_purchase_segment_product_list["{}".format(row[5])]+1 #該品項或商品的count+1
            except:
                a_purchase_segment_product_list.update({"{}".format(row[5]) : 1}) 

            #BehaviorType
            try:
                a_purchase_segment_behavior_type["{}".format(row[3])] = a_purchase_segment_behavior_type["{}".format(row[3])]+1
            except:
                a_purchase_segment_behavior_type.update({"{}".format(row[3]) : 1})

            try:
                a_purchase_segment_traffic_source["{}".format(row[1])] = a_purchase_segment_traffic_source["{}".format(row[1])]+1
            except:
                a_purchase_segment_traffic_source.update({"{}".format(row[1]) : 1})


        #算purchase裡，session的跳動  => 11231為跳三次 
        
            if(row[2]!=file.values[i-1][2]): # i-1為上一行 ,意思即是session跳動 i.e.如果session不等於上一個session的number
#                 #if(row[2] + 1 != file.values[i-1][2] ):
#                 session_count_index.append(i-1)
                
                
#                 if(file.values[i+1][3]!='Offline'):
#                     if(file.values[i+1][3]!='Purchase'):
#                         session_count_index.append(i)
                
                
                if((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds()>= 1800 or row[4]!=file.values[i-1][4]):  #if 超過1800秒 or 使用來源(web, app)不同:
                    a_purchase_segment_SessionCount += 1 
                    
#                     elif(row[0].split(" ")[0] !=file.values[i-1][0].split(" ")[0] and row[4]!=file.values[i-1][4]):           #elif 換天flag==True and session數兩者不相同:
#                          a_purchase_segment_SessionCount += 1
                elif(row[0].split(" ")[0] ==file.values[i-1][0].split(" ")[0] and row[4]!=file.values[i-1][4]):            #elif 換天flag==False and session數兩者不相同:
                    a_purchase_segment_SessionCount += 1

            
#                 #算SessionSeriesCount
                if(row[2]!=file.values[i-1][2]+1 ): # 如果sessionnumber不連續        
                    if((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds()>= 1800 and row[0].split(" ")[0] ==file.values[i-1][0].split(" ")[0]):
                        a_purchase_segment_SessionSeriesCount += 1
#                         session_series_count_index.append(i-1) # 儲存一個session series最後一個row之index
                
#             else: #如果session不跳動
#                 if(row[2]==file.values[i+1][2] and row[2]==1): # 
#                     session_series_count_index.append(i-1)
            
        #如果該筆資料為purchase，把之前計算之數值放入dict,並且全部重新計算
        elif(row[3]=='Purchase' or row[3]=='Offline'): #就是else，為purchase或offline的情況
#             if(file.values[i-1][3]!='Purchase'):
#                 if(file.values[i-1][3]!='Offline'):
#                     if(i != 0):
#                         session_count_index.append(i-1)
            
            
            

            
            if(a_purchase_segment_SessionCount!=0): #把之前count的東西全部裝在一個list
                #計算session,series total time
#                 for index in range(0,len(session_count_index),2):
#                     print(session_count_index)
#                     print(pd.to_datetime(file.values[session_count_index[index+1]][0]))
#                     print(pd.to_datetime(file.values[session_count_index[index]][0]))
#                     #a_purchase_segment_SessionCount_totaltime += (pd.to_datetime(file.values[index+1][0]) - pd.to_datetime(file.values[index][0])).total_seconds()
#                     #print(a_purchase_segment_SessionCount_totaltime)
                
#                 for index in session_series_count_index[::2]:
#                     a_purchase_segment_SessionSeriesCount_totaltime += (pd.to_datetime(file.values[index+1][0]) - pd.to_datetime(file.values[index][0])).total_seconds()                

                
                before_purchase_counts.append({'segment_session_count' : a_purchase_segment_SessionCount,
                                               'segment_session_series_count' : a_purchase_segment_SessionSeriesCount,
                            'source_type' : a_purchase_segment_source_type,
                             'product_list' : a_purchase_segment_product_list,
                            'behavior_type' : a_purchase_segment_behavior_type,
                            'traffic_source' : a_purchase_segment_traffic_source,
#                             'session_count_time' : a_purchase_segment_SessionCount_totaltime,
#                             'session_series_count_time' : a_purchase_segment_SessionSeriesCount_totaltime,
                            'traffic_count' : a_purchase_segment_row
                                               
                                               
                            })

#             a_purchase_segment_SessionCount_totaltime = 0
#             a_purchase_segment_SessionSeriesCount_totaltime = 0
            a_purchase_segment_source_type = {}
            a_purchase_segment_product_list = {}
            a_purchase_segment_row = 0
            a_purchase_segment_SessionCount = 0
            a_purchase_segment_SessionSeriesCount = 1
            a_purchase_segment_behavior_type ={}
            a_purchase_segment_traffic_source ={}
            a_purchase_segment_row = 0
            
            
            #Purchase單 計算

            if( row[0] != file.values[i-1][0]): #為purchase或offline， 且與上一row時間不同
                if(row[0] != file.values[i+1][0] ): #已知下一筆資料不為同一筆訂單=>只購買一個單獨商品的訂單
                    a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    a_purchase_order['purchase_amount'] = float(row[5].split(" ")[1])
                    a_purchase_order['discount_amount'] = float(row[5].split(" ")[2])         
                    purchase_orders.append({'purchaseOrder{}{}'.format(row[3],purchase_order_serial_number) : a_purchase_order})
                    purchase_amount=0
                    discount_amount=0
                    a_purchase_order = {}
                    purchase_order_serial_number += 1                    
                    continue
                else: # 已知下一筆資料為同筆訂單
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount                    



#                 elif(row[0] != file.values[i+1][0] ): #已知下一筆資料不為同一筆訂單=>只購買一個單獨商品的訂單
#                     a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
#                     a_purchase_order['purchase_amount'] = float(row[5].split(" ")[1])
#                     a_purchase_order['discount_amount'] = float(row[5].split(" ")[2])         
#                     purchase_orders.append({'purchaseOrder{}{}'.format(row[3],purchase_order_serial_number) : a_purchase_order})
#                     a_purchase_order = {}
#                     purchase_order_serial_number += 1



            else: #如果是 purchase或offline,且時間與上一row時間相同
                if(row[0] == file.values[i+1][0]): #下一row為同筆訂單
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount
                else: #同一訂單最後一個row的情況

                    #算最後一次的金額
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount

                    #add 商品or種類 count
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1}) 


                    purchase_orders.append({'purchaseOrder{}{}'.format(row[3],purchase_order_serial_number) : a_purchase_order})
                    #歸零
                    a_purchase_order = {}
                    purchase_amount=0
                    discount_amount=0
                    purchase_order_serial_number += 1
                    
                    
                    
#         if(i+2 == len(file.index)):
#             if(file.values[i+1][0]==0): # 表示讀到整個資料表最後一row了
#                 if(row[3]!='Purchase' and row[3]!='Offline'):
#                     session_count_index.append(i)
                
                

            
            
            
    return before_purchase_counts,purchase_orders



In [528]:
a = purchaseBeforeCount(file)
a

([{'segment_session_count': 3,
   'segment_session_series_count': 1,
   'source_type': {'APP': 24},
   'product_list': {'35222': 1,
    '33652': 1,
    '183450': 1,
    '4143210': 1,
    '4678808': 1,
    '4700585': 1,
    '4678939': 2,
    '4735719': 1,
    '4721435': 1,
    '86270': 1,
    '4715275': 1,
    '4648175': 2,
    '4721564': 1,
    '4679507': 1,
    '4678921': 2,
    '4678821': 1,
    '4656011': 1,
    '4700252': 1,
    '4636310': 1,
    '4567709': 1,
    '4246981': 1},
   'behavior_type': {'ViewSalePageCategory': 4, 'ViewSalePage': 20},
   'traffic_source': {'Direct': 24},
   'traffic_count': 24},
  {'segment_session_count': 2,
   'segment_session_series_count': 1,
   'source_type': {'APP': 14},
   'product_list': {'51876': 1,
    '4758407': 3,
    '4814269': 2,
    '4814395': 1,
    '4814425': 2,
    '182295': 1,
    '231141': 1,
    '4835216': 1,
    '4835882': 1,
    '35222': 1},
   'behavior_type': {'ViewSalePageCategory': 4, 'ViewSalePage': 10},
   'traffic_source': 